In [7]:
%pylab inline
import pandas as pd
import numpy as np
import time
from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

Populating the interactive namespace from numpy and matplotlib


In [2]:
def read_csv(day_trade):
    data_up = []
    data_down = []
    path = '/home/bigdatas16/SGX-OrderBook-Tick-Data-Trading-Strategy-/Train_Test_Data/ML_data_2014'
    for j,i in enumerate(day_trade):
        for k in range(0,len(i),1):
            path_up = path + '_' + str(j+1) + '_' + str(i[k]) + '_' + 'UP' + '.csv'
            path_down = path + '_' + str(j+1) + '_' + str(i[k]) + '_' + 'DOWN' + '.csv'
            data_up.append(pd.read_csv(path_up))
            data_down.append(pd.read_csv(path_down))
            #print path_down
    return data_up,data_down

In [5]:
day_trade = [[2,3,6,7,8,10,13,14,15,16,17,20,21,22,23,24,27,28,29,30],\
             [7,10,11,12,13,17,18,19,21,24,25,26,27],\
             [3,4,5,6,7,10,11,13,14,17,18,19,20,24,25,26,27,31]]

In [6]:
data_2014_up, data_2014_down = read_csv(day_trade)

In [23]:
latest_sec = 60 * 30
pred_sec = 5

Accuracy_Mean_Day = []
Accuracy_Std_Day = []
Feature_Importance = []
Feature_Importance_Day_one = []
Feature_Importance_Day_two = []
Feature_Importance_Day_three = []
Feature_Importance_Day_four = []
Feature_Importance_Day_five = []
Predict_Values_Day = []
CV_Best_Mean_Acc_Day = []
for day in range(0,1,1):
    Accuracy = []
    Feature_Importance_one = []
    Feature_Importance_two = []
    Feature_Importance_three = []
    Feature_Importance_four = []
    Feature_Importance_five = []
    predict_values = []
    CV_Best_Mean_Acc = []
    for i in range(0,9000-latest_sec-pred_sec,pred_sec):

        print '----------------------------Rolling Window Time = %s----------------------------'%(i/pred_sec)
        data_train = data_2014_up[day][i:i+latest_sec]
        X_train = data_train.drop(["0"],axis=1)
        y_train = data_train['0']

        data_test = data_2014_up[day][i+latest_sec:i+latest_sec+pred_sec]
        X_test = data_test.drop(["0"],axis=1)
        y_test = data_test['0']

        # hyper-parameter
        model_grid = {"base_estimator__criterion" : ["entropy"],
                      "base_estimator__max_depth": [None],
                      "base_estimator__min_samples_leaf" : [5],
                      "base_estimator__min_samples_split" : [2],
                      "base_estimator__max_features" : [None]
                     }
        
        Dtc = DecisionTreeClassifier()
        model = AdaBoostClassifier(base_estimator = Dtc,n_estimators = 10)
        
        # cv
        Grid = GridSearchCV(model, model_grid,cv = 5, scoring = 'accuracy')
        %time Grid.fit(X_train,y_train) 
        print Grid.best_params_
        print 'CV Best Score = %s'%(Grid.best_score_)
        CV_Best_Mean_Acc.append(Grid.best_score_)
        # training & testing
        model.set_params(**Grid.best_params_)
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        print 'Prediction latest 15 second = %s'%(predictions)
        predict_values.append(predictions.tolist())
        acc = metrics.accuracy_score(y_test, predictions)
        print 'Accuracy = %s'%(acc)
        Accuracy.append(acc)
        feature_imp = dict(zip([i for i in range(0,64,1)],model.feature_importances_))
        
        Top_five = sorted(feature_imp.items(),key = lambda x : x[1] , reverse=True)[0:5]
        print 'Top five importance features = %s'%(Top_five)
        
        Feature_Importance_one.append(Top_five[0][0])
        Feature_Importance_two.append(Top_five[1][0])
        Feature_Importance_three.append(Top_five[2][0])
        Feature_Importance_four.append(Top_five[3][0])
        Feature_Importance_five.append(Top_five[4][0])
        
    print 'Day = %s, mean = %s, std = %s'%(day,np.mean(Accuracy),np.std(Accuracy))
    
    Accuracy_Mean_Day.append(Accuracy)
    Accuracy_Std_Day.append(Accuracy)
    CV_Best_Mean_Acc_Day.append(CV_Best_Mean_Acc)
    Feature_Importance_Day_one.append(Feature_Importance_one)
    Feature_Importance_Day_two.append(Feature_Importance_two)
    Feature_Importance_Day_three.append(Feature_Importance_three)
    Feature_Importance_Day_four.append(Feature_Importance_four)
    Feature_Importance_Day_five.append(Feature_Importance_five)
    Predict_Values_Day.append(predict_values)
    

----------------------------Rolling Window Time = 0----------------------------
CPU times: user 1.7 s, sys: 7.12 ms, total: 1.71 s
Wall time: 2.31 s
{'base_estimator__min_samples_split': 2, 'base_estimator__criterion': 'entropy', 'base_estimator__max_depth': None, 'base_estimator__min_samples_leaf': 5, 'base_estimator__max_features': None}
CV Best Score = 0.733888888889
Prediction latest 15 second = [ 1.  1.  1.  1.  1.]
Accuracy = 1.0
Top five importance features = [(48, 0.086710590001048693), (62, 0.085261029159579185), (63, 0.083308856146032409), (57, 0.081735991929801979), (29, 0.062088372221611662)]
----------------------------Rolling Window Time = 1----------------------------
CPU times: user 1.61 s, sys: 3.98 ms, total: 1.61 s
Wall time: 1.69 s
{'base_estimator__min_samples_split': 2, 'base_estimator__criterion': 'entropy', 'base_estimator__max_depth': None, 'base_estimator__min_samples_leaf': 5, 'base_estimator__max_features': None}
CV Best Score = 0.780555555556
Prediction lat

KeyboardInterrupt: 

In [20]:
Grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': None,
 'base_estimator__max_features': None,
 'base_estimator__min_samples_leaf': 5,
 'base_estimator__min_samples_split': 2}

In [16]:
mean(Accuracy),std(Accuracy)

(0.91625615763546797, 0.24044657425906688)